In [11]:
import sys

sys.path.append("../")

In this file we'll compare all retrievers.
We'll check how each of them compares to each other,
We'll get the best 10 retrievers that will later be used for our RAG.

To test all retrievers:
1. Get all dense, sparse and hybrid retrievers
2. Prompt and get top 10 results for each retriever
3. Calculate metrics

REMEMBER TO CACHE WHAT YOU CAN!

In [15]:
from elasticsearch import Elasticsearch
from qdrant_client import QdrantClient
from cache.cache import Cache

qdrant_client = QdrantClient(host="localhost", port=6333)
es_client = Elasticsearch(
    hosts=["http://localhost:9200"],
)
cache = Cache()

In [17]:
from typing import Dict
from common.names import INDEX_NAMES
from repository.es_repository import ESRepository

es_repositories: Dict[str, ESRepository] = {}

for index_name in INDEX_NAMES:
    es_repositories[index_name] = ESRepository(es_client, index_name, cache)

In [18]:
from common.utils import replace_slash_with_dash

es_repositories["polish_whitespace_index"].find(
    "Platforma Obywatelska", replace_slash_with_dash(f"{"ipipan/polqa"}-{"character-500"}")
)

{}